In [3]:
pip install optuna 

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sklearn.model_selection import train_test_split
import optuna 
from sklearn.preprocessing import StandardScaler
import pandas as pd 

In [5]:
df=pd.read_csv("C:\\Users\\DELL\\Downloads\\diabetes.csv")

In [6]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
## problem there is some missing value which is represented bu 0 in insulin,and mostly some more columns  

In [8]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [9]:
import numpy as np 

In [10]:
## so what we can do is repace 0 with mean of that column 
columns_null = ["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]
df[columns_null]=df[columns_null].replace(0,np.nan)
df[columns_null]=df[columns_null].fillna(df[columns_null].mean())


                                  

## split into x and y 
make train and test split 
standard scaler

In [11]:
X=df.drop(columns=["Outcome"],axis=1)
y=df["Outcome"]

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [49]:
scale=StandardScaler()
X_trains=scale.fit_transform(X_train)
X_tests=scale.transform(X_test)

In [14]:
X_train.shape

(614, 8)

In [15]:
X_test.shape

(154, 8)

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [17]:
from sklearn.model_selection import cross_val_score

In [18]:
def objective_class(trial):## hum ek objective function banayenge jisme hum hyperparameter tuning karenge 
    n_estimators=trial.suggest_int("n_estimators",50,250)
    max_depth=trial.suggest_int("max_depth",2,20)
    ## define our model 
    model=RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )
    ## perform 3 cross validation 
    score=cross_val_score(model,X_train,y_train,cv=3,scoring="accuracy").mean()
    return score

In [19]:
## make a study object 
study=optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler())
study.optimize(objective_class,n_trials=50)

[I 2025-12-06 17:41:23,367] A new study created in memory with name: no-name-e215dddc-93db-424f-9bb2-fb3d79e6a7f2
[I 2025-12-06 17:41:24,370] Trial 0 finished with value: 0.780113183484776 and parameters: {'n_estimators': 164, 'max_depth': 16}. Best is trial 0 with value: 0.780113183484776.
[I 2025-12-06 17:41:25,761] Trial 1 finished with value: 0.7703331739199745 and parameters: {'n_estimators': 200, 'max_depth': 19}. Best is trial 0 with value: 0.780113183484776.
[I 2025-12-06 17:41:26,693] Trial 2 finished with value: 0.7687310696636378 and parameters: {'n_estimators': 134, 'max_depth': 8}. Best is trial 0 with value: 0.780113183484776.
[I 2025-12-06 17:41:27,346] Trial 3 finished with value: 0.7687230989956958 and parameters: {'n_estimators': 100, 'max_depth': 19}. Best is trial 0 with value: 0.780113183484776.
[I 2025-12-06 17:41:28,731] Trial 4 finished with value: 0.7719591901801371 and parameters: {'n_estimators': 156, 'max_depth': 19}. Best is trial 0 with value: 0.7801131834

In [20]:
study.best_trial.values

[0.7833891280089271]

In [21]:
study.best_trial.params

{'n_estimators': 113, 'max_depth': 16}

In [22]:
best_model=RandomForestClassifier(** study.best_trial.params,random_state=42)
best_model.fit(X_train,y_train)
y_pred=best_model.predict(X_test)
accuracy_score(y_test,y_pred)


0.7597402597402597

In [23]:
## we have acheived accuracy of 75 percent 

In [24]:
## now lets use some diff sampler 
def objective_class(trial):
    n_estimators=trial.suggest_int("n_estimators",50,250)
    max_depth=trial.suggest_int("max_depth",2,20)
    model=RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )
    score=cross_val_score(model,X_train,y_train,cv=3,scoring="accuracy").mean()
    return score

In [25]:
study=optuna.create_study(direction="maximize",sampler=optuna.samplers.RandomSampler())
study.optimize(objective_class,n_trials=50)

[I 2025-12-06 17:42:15,748] A new study created in memory with name: no-name-173c0882-4099-47c8-a8d6-4b4f2d788e87
[I 2025-12-06 17:42:17,202] Trial 0 finished with value: 0.7735931771082417 and parameters: {'n_estimators': 244, 'max_depth': 19}. Best is trial 0 with value: 0.7735931771082417.
[I 2025-12-06 17:42:18,378] Trial 1 finished with value: 0.7768611509644509 and parameters: {'n_estimators': 240, 'max_depth': 16}. Best is trial 1 with value: 0.7768611509644509.
[I 2025-12-06 17:42:19,281] Trial 2 finished with value: 0.7605770763589988 and parameters: {'n_estimators': 186, 'max_depth': 4}. Best is trial 1 with value: 0.7768611509644509.
[I 2025-12-06 17:42:19,804] Trial 3 finished with value: 0.7736011477761836 and parameters: {'n_estimators': 117, 'max_depth': 6}. Best is trial 1 with value: 0.7768611509644509.
[I 2025-12-06 17:42:20,830] Trial 4 finished with value: 0.7784791965566714 and parameters: {'n_estimators': 185, 'max_depth': 5}. Best is trial 4 with value: 0.7784791

In [26]:
study.best_trial.values

[0.7833811573409851]

In [27]:
study.best_trial.params

{'n_estimators': 132, 'max_depth': 13}

In [28]:
best_model=RandomForestClassifier(n_estimators=146,max_depth=13,random_state=42)
best_model.fit(X_train,y_train)
y_pred=best_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.7532467532467533

In [29]:
from sklearn import svm
from sklearn.svm import SVC


In [41]:
def function(trial):
    category=trial.suggest_categorical("category",["SVM","RandomForest"])
    if category=="SVM":
        c=trial.suggest_float("C",0.1,100,log=True)
        gamma=trial.suggest_categorical("gamma",["scale","auto"])
        kernel=trial.suggest_categorical("kernel",["linear","rbf","poly"])
        model=SVC(C=c,gamma=gamma,kernel=kernel,random_state=42)
        pipeline=Pipeline([
            ("scaler",StandardScaler()),
            ("svc_model",model)
        ])
    elif category=="RandomForest":
        n_estimators=trial.suggest_int("n_estimators",50,250)
        max_depth=trial.suggest_int("max_depth",2,20)
        model=RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42
        )
        pipeline=Pipeline([
            ("rf_model",model)
        ])
    score=cross_val_score(pipeline,X_train,y_train,cv=3,scoring="accuracy").mean()
    return score 

        

In [43]:
from sklearn.pipeline import Pipeline


In [44]:
study=optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler())
study.optimize(function,n_trials=50)
                          

[I 2025-12-06 18:03:08,259] A new study created in memory with name: no-name-33dd13b6-a9ee-4d57-b019-270938ae0b10
[I 2025-12-06 18:03:11,261] Trial 0 finished with value: 0.7736091184441256 and parameters: {'category': 'RandomForest', 'n_estimators': 196, 'max_depth': 15}. Best is trial 0 with value: 0.7736091184441256.
[I 2025-12-06 18:03:12,777] Trial 1 finished with value: 0.7768691216323927 and parameters: {'category': 'RandomForest', 'n_estimators': 120, 'max_depth': 17}. Best is trial 1 with value: 0.7768691216323927.
[I 2025-12-06 18:03:15,012] Trial 2 finished with value: 0.7719432488442531 and parameters: {'category': 'RandomForest', 'n_estimators': 162, 'max_depth': 6}. Best is trial 1 with value: 0.7768691216323927.
[I 2025-12-06 18:03:18,156] Trial 3 finished with value: 0.7687071576598119 and parameters: {'category': 'RandomForest', 'n_estimators': 169, 'max_depth': 7}. Best is trial 1 with value: 0.7768691216323927.
[I 2025-12-06 18:03:18,340] Trial 4 finished with value:

In [45]:
best_trial=study.best_trial
best_trial.values

[0.7866331898613104]

In [46]:
best_trial = study.best_trial
best_params = best_trial.params
print(best_params)


{'category': 'RandomForest', 'n_estimators': 160, 'max_depth': 13}


## ended up with a accuracy 0f 79 percent 4 percent increase 

In [52]:
best_trial.params

{'category': 'RandomForest', 'n_estimators': 160, 'max_depth': 13}

In [34]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_category,params_gamma,params_kernel,params_max_depth,params_n_estimators,state
0,0,0.780113,2025-12-06 17:43:00.377504,2025-12-06 17:43:00.800285,0 days 00:00:00.422781,NaN,RandomForest,NaN,NaN,16.0,62.0,COMPLETE
1,1,0.767081,2025-12-06 17:43:00.814349,2025-12-06 17:43:01.699952,0 days 00:00:00.885603,NaN,RandomForest,NaN,NaN,7.0,173.0,COMPLETE
2,2,0.695465,2025-12-06 17:43:01.699952,2025-12-06 17:43:01.836223,0 days 00:00:00.136271,81.050430,SVM,scale,rbf,NaN,NaN,COMPLETE
3,3,0.758959,2025-12-06 17:43:01.836223,2025-12-06 17:43:02.165044,0 days 00:00:00.328821,NaN,RandomForest,NaN,NaN,9.0,62.0,COMPLETE
4,4,0.768731,2025-12-06 17:43:02.165044,2025-12-06 17:43:03.181607,0 days 00:00:01.016563,NaN,RandomForest,NaN,NaN,7.0,227.0,COMPLETE
5,5,0.763829,2025-12-06 17:43:03.181607,2025-12-06 17:43:03.698712,0 days 00:00:00.517105,NaN,RandomForest,NaN,NaN,8.0,106.0,COMPLETE
6,6,0.768699,2025-12-06 17:43:03.698712,2025-12-06 17:43:03.714546,0 days 00:00:00.015834,0.414726,SVM,auto,linear,NaN,NaN,COMPLETE
7,7,0.754065,2025-12-06 17:43:03.714546,2025-12-06 17:43:04.215271,0 days 00:00:00.500725,NaN,RandomForest,NaN,NaN,4.0,116.0,COMPLETE
8,8,0.737805,2025-12-06 17:43:04.230711,2025-12-06 17:43:04.265188,0 days 00:00:00.034477,2.659528,SVM,scale,rbf,NaN,NaN,COMPLETE
9,9,0.781739,2025-12-06 17:43:04.265188,2025-12-06 17:43:05.316945,0 days 00:00:01.051757,NaN,RandomForest,NaN,NaN,11.0,218.0,COMPLETE


In [59]:
import joblib
joblib.dump(modelss,"best_diabetes_models.pkl")

['best_diabetes_models.pkl']

In [58]:
modelss=RandomForestClassifier(
    n_estimators=160,
    max_depth=13,
    random_state=42
)
modelss.fit(X_train,y_train)
y_pred=modelss.predict(X_test)
accuracy_score(y_test,y_pred)

0.7532467532467533

In [55]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)


array([[79, 20],
       [18, 37]], dtype=int64)

In [57]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')